In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render li, div.text_cell_render p, code{font-size:22pt; line-height:40px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:red">ch2.Ollama_LLM활용의 기본 개념(LangChain)</span>
# 1.  LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM 이용
성능은 GPT, CLaude 같은 모델보다 떨어지나, 개념설명을 위해 open source 모델 사용

### ollama.com 다운로드 -> 설치 -> 모델 pull
- cmd창이나 powershell 창에 ollama pull deepseek-r1:1.5b
https://docs.langchain.com/oss/python/integrations/chat/ollama

In [5]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:1.5b")
result = llm.invoke("What is the captital of Korea?")
result

AIMessage(content='The capital of Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-09T02:05:09.771942Z', 'done': True, 'done_reason': 'stop', 'total_duration': 311237600, 'load_duration': 100486800, 'prompt_eval_count': 11, 'prompt_eval_duration': 15993300, 'eval_count': 12, 'eval_duration': 175396000, 'logprobs': None, 'model_name': 'deepseek-r1:1.5b', 'model_provider': 'ollama'}, id='lc_run--019b00da-f9d3-7992-a202-21a1966fa175-0', usage_metadata={'input_tokens': 11, 'output_tokens': 12, 'total_tokens': 23})

### 모델 pull
- cmd창이나 powershell창(window키+R에서 powershell)에서 ollama pull llama3.2:1b

In [13]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of Korea?")
result

AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that there are also two other cities that serve as major centers: Busan in the south and Incheon in the west.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T02:22:50.7896232Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1154751300, 'load_duration': 125474600, 'prompt_eval_count': 32, 'prompt_eval_duration': 39557100, 'eval_count': 40, 'eval_duration': 939153800, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b00eb-2722-7dc1-b4f3-2a262bfc78fe-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [14]:
result.content

"The capital of South Korea is Seoul. However, it's worth noting that there are also two other cities that serve as major centers: Busan in the south and Incheon in the west."

In [15]:
result = llm.invoke("일본 수도가 어디예요?")
result.content

'Japan의 수도는 Tokyo입니다. Tokyoは日本最も나icing도高い都市です。'

## 2) openai 활용
- pip install langchain-openai
- https://auth.openai.com/log-in

In [3]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# os.getenv("OPENAI_API_KEY")

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano",
                #api_key=os.getenv("OPENAI_API_KEY")
                )
result = llm.invoke("What is the capitaal of Korea? Return the name of the city only.")

In [5]:
result

AIMessage(content='Seoul', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 203, 'prompt_tokens': 22, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CkkVef1qFLb60opVXts7a5APihm3W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0192-050b-7370-a2e9-717f6b021437-0', usage_metadata={'input_tokens': 22, 'output_tokens': 203, 'total_tokens': 225, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [7]:
# Azure : OPENAI_API_VERSION키값
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model="gpt-5-nano")

# 2. 렝체인 스타일로 프롬프트 작성
- 프롬프트 : llm호출시 쓰는 질문

In [10]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
# llm.invoke(0)
# PromptValue, str, BaseMessages리스트

## 1) 기본 프롬프트 템플릿 사용
- PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [16]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)
# country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":"Korea"})
print(prompt)
llm.invoke(prompt)

text='What is the capital of Korea?'


AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Seoul as its own capital, and the two countries have a long-standing rivalry. But for official purposes, Seoul is recognized as the capital of South Korea.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T05:47:44.6780933Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2659583300, 'load_duration': 1885352500, 'prompt_eval_count': 32, 'prompt_eval_duration': 40899100, 'eval_count': 51, 'eval_duration': 671917900, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01a6-b841-7863-84ba-f4047df89196-0', usage_metadata={'input_tokens': 32, 'output_tokens': 51, 'total_tokens': 83})

## 2) 메세지 기반 프롬프트 작성
- BaseMessage 리스트
- BaseMessage 상속받은 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage
- vscode에서 ctrl+shift+p : python:select interpreter입력 -> python환경선택
- vscode에서 커널 선택

In [18]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of Korea?")
]
llm.invoke(message_list)

AIMessage(content='The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:04:19.061392Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2441705100, 'load_duration': 1911786300, 'prompt_eval_count': 86, 'prompt_eval_duration': 81113100, 'eval_count': 17, 'eval_duration': 434927200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01b5-e569-7e32-9efe-c3ca13a09ae6-0', usage_metadata={'input_tokens': 86, 'output_tokens': 17, 'total_tokens': 103})

In [19]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of {country}?")
]
print(message_list)

[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Rome.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of {country}?', additional_kwargs={}, response_metadata={})]


## 3) ChatPromptTemplate 사용
- BaseMessage리스트 -> 튜플리스트

In [26]:
# 위의 BaseMessage 리스트를 수정
# PromptTemplate : 프롬프트에 변수포함, 
# ChatPromptTemplate : SystemPrompt설정(페르소나), few shot설정, 변수포함
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpfull assistant!"),
    ('human', "What is the capital of Italy?"), # 모범질문
    ("ai", "The capital of Italy is Rome."),    # 모범답안
    ('human', "What is the capital of France?"), # 모범질문
    ("ai", "The capital of France is Paris."),    # 모범답안
    ("human", "What is the capital of {country}?")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPrompt_template.invoke({"country": country})
#print("프롬프트 : ", prompt, type(prompt))
result = llm.invoke(prompt)
result.content

어느 나라 수도가 궁금하세요Korea


'The capital of North Korea and the Demilitarized Zone (DMZ) between North and South Korea is Pyongyang. The capital of South Korea is Seoul.'

In [28]:
chatPromptTemplate = ChatPromptTemplate.from_messages([
    ("system", "당신은 대한민국 정보 전문 도우미입니다"),
    ("human", "{country}의 수도가 어디예요!")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPromptTemplate.invoke({"country":country})
# print(prompt)
result = llm.invoke(prompt)
print(result.content)

어느 나라 수도가 궁금하세요Korea
한국의 수도는 Seoul입니다.


In [29]:
result

AIMessage(content='한국의 수도는 Seoul입니다.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:38:20.5815546Z', 'done': True, 'done_reason': 'stop', 'total_duration': 520123600, 'load_duration': 135716400, 'prompt_eval_count': 44, 'prompt_eval_duration': 185103700, 'eval_count': 8, 'eval_duration': 187573600, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01d5-139b-7f70-8a40-1347ad7fa0fc-0', usage_metadata={'input_tokens': 44, 'output_tokens': 8, 'total_tokens': 52})

# 3. 답변 형식 컨트롤하기
- llm.invoke()의 결과는 AIMessage() -> string이나 json, 객체 : OutputParser이용

## 1) 문자열 출력 파서 이용
- StrOutputParser를 사용하여 LLM출력(AIMessage)을 단순 문자열로 변환

In [35]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
# 명시적인 지시하상이 포함된 프롬프트
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
# 프롬프트 템플릿에 값 주입
prompt = prompt_template.invoke({"country": "Korea"})
# print(prompt)
ai_message = llm.invoke(prompt)
# print(ai_message)
# 문자열 출력 파서를 이용하여 llm응답(AIMessage객체)을 단순 문자열로 변환
output_parser = StrOutputParser()
result = output_parser.invoke(ai_message)
result

'Seoul'

In [36]:
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

In [47]:
# 변수설정, system, few shot 지정
chat_prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant with expertise in South Korea."),
    ('human', "What is the capital of Italy?"), # 모범질문
    ("ai", "Rome"),    # 모범답안
    ('human', "What is the capital of France?"), # 모범질문
    ("ai", "Paris"),    # 모범답안
    ("human", "What is the capital of {country}? Return the name of the city only.")#Return the name of the city only
])
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(chat_prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2)Json 출력 파서 이용
- {'name':'홍길동', 'age':22}

In [55]:
from langchain_core.output_parsers import JsonOutputParser
country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
prompt = country_detail_prompt.invoke({"country":"Korea"})
ai_message = llm.invoke(prompt)
# print(ai_message.content)
output_parser = JsonOutputParser()
json_result = output_parser.invoke(ai_message)
print(json_result, type(json_result))

{'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'Korean won'} <class 'dict'>


In [54]:
output_parser.invoke(llm.invoke(country_detail_prompt.invoke({"country":"Japen"})))

{'capital': 'Tokyo',
 'population': 128745561,
 'language': 'Japanese',
 'currency': ' yen'}

## 3) 구조화된 출력 사용
- Pydantic 모델을 사용하여 LLM출력을 구조화된 형식으로 받기(JsonParser보다 훨씬 안정적)
- Pydantic : 데이터유효성검사, 설정관리를 간편하게 해주는 라이브러리

In [62]:
class User:
    def __init__(self, id, name, is_active=True):
        self.id   = id
        self.name = name
        self.is_active = is_active
user = User("1", "홍길동", False)
print(user)

In [67]:
from pydantic import BaseModel, Field
class User(BaseModel):
    # gt=0:id>0 / ge=0:id>=0 / lt=0:id<0 / le=0:id<=0
    id:int   = Field(gt=0,         description="id")
    name:str = Field(min_length=2, description="name")
    is_active:bool = Field(default=True, description="id활성화 여부")
user = User(id="1", name="홍길동")
print(user)

id=1 name='홍길동' is_active=True


In [71]:
country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}.
    - Capital
    - Population
    - Language
    - Currency
    Return in JSON format and return the JSON dictionary only""",
    input_variables=["country"]
)
class CountryDetail(BaseModel): #description: 더 정확한 출력 유도
    capital:str  = Field(description="the capital of the country")
    population:int = Field(description="the population of the country")
    language:str = Field(description="the language of the country")
    currency:str = Field(description="the currency of the country")
# 출력 형식 파서 + LLM
structedllm = llm.with_structured_output(CountryDetail)
# llm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info = structedllm.invoke(country_detail_prompt.invoke({"country":"Korea"}))
info

CountryDetail(capital='Seoul', population=51000000, language='Korean', currency='Korean won')

In [72]:
type(info)

__main__.CountryDetail

In [73]:
info.capital, info.population, info.language, info.currency

('Seoul', 51000000, 'Korean', 'Korean won')

In [77]:
print("info를 json 스타일로 :", info.model_dump_json())
print("info를 dict로 :", info.model_dump())
print("info를 dict로 :", info.__dict__)

info를 json 스타일로 : {"capital":"Seoul","population":51000000,"language":"Korean","currency":"Korean won"}
info를 dict로 : {'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'Korean won'}
info를 dict로 : {'capital': 'Seoul', 'population': 51000000, 'language': 'Korean', 'currency': 'Korean won'}


In [78]:
type(info.model_dump())

dict

# 4. LCEL을 활용한 렝체인 생성하기
## 1) 문자열 출력 파서 사용
- invoke : Runnable에 있는 함수
- StrOutputParser, ChatOllama, PromptTemplate등은 모두 Runnable로부터 상속 받음.

In [79]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.2:1b",
                temperature=0) # 일관된 답변(보수적인 답변)
prompt_template = PromptTemplate(
    template = "What is the capital of {country}. Retrun the name of the city only.",
    input_variables = ["country"]
)
output_parser = StrOutputParser() # AIMessage()를 Str변환
output_parser.invoke(llm.invoke(prompt_template.invoke({"country":"Korea"})))

'Seoul'

## 2) LCEL을 사용한 간단한 체인 구성
- 파이프연산자(|) 이용

In [80]:
# 프롬프트 템플릿 -> llm -> 출력파서를 연결하는 체인 생성
capital_chain = prompt_template | llm | output_parser
# 생성된 체인 invoke
capital_chain.invoke({"country":"Korea"})

'Seoul'

## 3) 복합체인 구성
- 여러 단계의 추론이 필요한 경우(체인 연결)

In [81]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""Guess the name of the country based on the following informat:
    {information}
    Return the name of the country only""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "This country is very famous for its wine"})))

'Italy.'

In [84]:
# 나라명 추출 체인 생성
country_chain = country_prompt | llm | output_parser
country_chain.invoke({"information":"This country is very famous for its wine"})

'Italy.'

In [85]:
# 복합체인 : 나라설명 -> 나라명(country_chain)
#                      나라명 -> 수도(capital_chain)
final_chain = country_chain | capital_chain
final_chain.invoke({"information":"This country is very famous for its wine"})

'Rome'

In [86]:
# 복합체인 : information -> country_chain -> (나라명을 country) -> capital_chain
from langchain_core.runnables import RunnablePassthrough
final_chain = {"information":RunnablePassthrough()} | \
                {"country":country_chain} | capital_chain

In [87]:
final_chain.invoke("This country is very famous for its wine")

'Rome'

- 한글 지원이 안 되는 모델은 렝체인 연결이 잘 안 됨

In [88]:
# 나라 설명 -> 나라이름
country_prompt = PromptTemplate(
    template="""다음의 {information} 설명을 보고 나라이름을 맞춰봐:
    {information}
    나라 이름만 한국어로 reutrn 해 줘""",
    input_variables=["information"]
)
output_parser.invoke(llm.invoke(country_prompt.invoke({"information":
                            "이 나라는 와인으로 유명해"})))

'이 나라는 와인으로 유명해\n\n1. 프랑스 - burgundy, merlot, cabernet sauvignon\n2. 이탈리아 - chianti, pinot noir, vermentino\n3. 스태프라니 - barolo, barbaresco, nebbusco\n4. 이스라엘 - merom, sharon, golan\n5. 미국 - california, napa valley, sonoma county'

# 5. 생성형 AI 평가: 나라명을 입력하면 그 나라의 제일 유명한 음식의 레시프를 출력하는 복합체인 구현

- 위 : 나라설명->나라이름->수도 (체인두개 연결)
- 나라이름 -> 그 나라에서 제일 유명한 음식 -> 레시피
- exaone3.5:2.4b, 
- llama3.1:405b(수능 언어영역 2위), llama3:70b(llama3.1:405b와 유사)
- llama3.2:3b

In [2]:
## 복합체인 : 나라명 -> 그 나라에서 제일 유명한 음식(food_chain)
##                     음식 -> 레시피(recipe_chain)

## 1) LLM 설정

In [4]:
from langchain_ollama import ChatOllama

# Temperature를 0으로 설정하여 일관된 답변 유도
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
)

## 2) 음식 추천 체인 구성

In [5]:
# 음식 이름 찾기

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

# 음식 이름만 반환하도록 명확하게 지시
food_prompt = PromptTemplate(
    template='''what is one of the most popular food in {country}? 
        Please return the name of the food only''', # token아끼자
    input_variables=['country']
)

# 음식 추천 체인 구성
food_chain = food_prompt | llm | StrOutputParser()
# 음식 추천 체인 실행
food_chain.invoke({'country': 'South Korea'})

'Bibimbap.'

## 3) 레시피 생성 체인 구성

In [7]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지를 통해 레시피 형식 지정
recipe_prompt = ChatPromptTemplate.from_messages([
    ('system', '''Provide the recipe of the food that the user wants. 
Please return the recipe only as a numbered list without any explanation'''), #input token보다 output token이 비쌈
    ('human', 'Can you give me the recipe for making {food}?')
])

# 레시피 생성 체인 구성
recipe_chain = recipe_prompt | llm | StrOutputParser()
# 레시피 생성 체인 실행
recipe_chain.invoke({'food': 'Bibimbap'})

"Here's a traditional Korean recipe for Bibimbap:\n\n1. Prepare the ingredients:\n\t* 8 oz. mixed vegetables (zucchini, carrots, bean sprouts)\n\t* 1 cup cooked white rice\n\t* 2 eggs, fried or poached\n\t* 1/4 cup chopped green onions\n\t* 1/4 cup toasted sesame seeds\n\t* 1/4 cup diced cucumber\n\t* 1/4 cup diced zucchini\n\t* 1/4 cup diced carrots\n\t* 2 tbsp. soy sauce\n\t* 1 tsp. Gochujang (Korean chili paste)\n\t* Salt and pepper to taste\n\t* Vegetable oil for frying\n\n2. Cook the rice:\n\t* Rinse the rice thoroughly and cook according to package instructions.\n\n3. Prepare the vegetables:\n\t* Cut the mixed vegetables into bite-sized pieces.\n\t* Slice the eggs into wedges.\n\n4. Assemble the Bibimbap:\n\t* Divide the cooked rice onto a plate or bowl.\n\t* Arrange the mixed vegetables, fried egg, green onions, sesame seeds, cucumber, and zucchini on top of the rice.\n\t* Drizzle with soy sauce and Gochujang.\n\t* Season with salt and pepper to taste.\n\n5. Serve immediately:\n

## 4) 체인 연결을 통한 최종 체인 구성

In [8]:
# 음식 추천과 레시피 생성을 연결한 최종 체인 구성
from langchain_core.runnables import RunnablePassthrough
final_chain = food_chain | recipe_chain
# final_chain = {"country":RunnablePassthrough()}| food_chain | recipe_chain
# final_chain = {"country":RunnablePassthrough()}|{'food': food_chain} | recipe_chain
# 국가 입력으로 대표 음식을 찾고, 해당 음식의 레시피까지 자동으로 생성
recipe = final_chain.invoke("Korea")
print(recipe)

Here's a traditional Korean recipe for tteok (also spelled as tteokguk or ramyeon):

**Ingredients:**

1. 200g glutinous rice flour
2. 1000ml water
3. 1/4 cup salt
4. 1 tablespoon sugar
5. 1/4 teaspoon ground black pepper
6. Filling ingredients:
   - 500g pork, diced
   - 1 onion, diced
   - 2 cloves garlic, minced
   - 1 cup kimchi, chopped
   - 1 egg, beaten
7. Optional: sesame oil, soy sauce, and green onions for garnish

**Instructions:**

1. In a large pot, combine glutinous rice flour, water, salt, sugar, and black pepper.
2. Bring the mixture to a boil over high heat, then reduce the heat to medium-low and simmer for 10 minutes.
3. Remove from heat and let cool slightly.
4. Knead the dough until it becomes pliable and smooth.
5. Divide the dough into small balls, about the size of a golf ball.
6. Roll out each ball into a thin sheet, about 1/16 inch thick.
7. Cut the sheets into long, thin strips, about 2 inches wide and 10-12 inches long.
8. Cook the tteok in boiling water for 